# To run this file, install Python packages: 
- requests
- tabulate

In [5]:
import json
import requests
from random import randrange
from tabulate import tabulate

# Year range
start_year = 2010
end_year = 2020
year = randrange(start_year, end_year)
game_season = str(year) + "-" + str((year + 1) % 100).rjust(2, "0")

# Game type
game_types = ["Regular Season", "Playoffs"]
game_type = game_types[randrange(len(game_types))]

# Game data
players_url = "https://api.pbpstats.com/get-games/nba"
games_params = {
    "Season": game_season,
    "SeasonType": game_type
}
response = requests.get(players_url, params=games_params)
response_json = response.json()["results"]

# Random a game
game_id= randrange(len(response_json))
game_overview = response_json[game_id]
print(json.dumps(game_overview, indent=2))

# Get game stats from the overview
print("Extracting game stats for game:", game_overview["GameId"], "...")
players_url = "https://api.pbpstats.com/get-game-stats"
games_params = {
    "GameId": game_overview["GameId"],
    "Type": "Player" # options are Player, Lineup and LineupOpponent
}
response = requests.get(players_url, params=games_params)
game_stats = response.json()['stats']
print("Done")

{
  "GameId": "0021200795",
  "Date": "2013-02-19",
  "HomeTeamId": "1610612762",
  "AwayTeamId": "1610612744",
  "HomePoints": 115,
  "AwayPoints": 101,
  "HomePossessions": 92,
  "AwayPossessions": 93,
  "HomeTeamAbbreviation": "UTA",
  "AwayTeamAbbreviation": "GSW"
}
Extracting game stats for game: 0021200795 ...
Done


## Some util functions:

In [36]:
# Stats to get
key_general = ["GamesPlayed"]
key_attack_stats = ["Fg2Pct", "Fg3Pct", "FtPoints", "FTA", "Turnovers", "TotalPoss"]
key_defend_stats = ["Steals", "Blocks", "Fouls"]
key_all_stats = key_general + key_attack_stats + key_defend_stats

# Uitil functions
def get_team_players(game_stats, team_type):
    team = []
    if team_type == "Away":
        stats = game_stats["Away"]["FullGame"]
    elif team_type == "Home":
        stats = game_stats["Home"]["FullGame"]

    for entity in stats:
        if entity["Name"] != "Team":
            team.append([entity["Name"], entity["Minutes"]])
    team.sort(key=lambda lst: lst[1], reverse=True)
    return team


def get_player_ids(team, team_type):
    if team_type == "Away":
        team_id = game_overview["AwayTeamId"]
    elif team_type == "Home":
        team_id = game_overview["HomeTeamId"]

    players_url = "https://api.pbpstats.com/get-team-players-for-season"
    players_params = {
        "Season": game_season,
        "SeasonType": game_type,
        "TeamId": team_id
    }
    players_response = requests.get(players_url, params=players_params)
    players = players_response.json()["players"]

    # "team" is list of list, each sub list starts with player name
    for player in team:
        for id, name in players.items():
            if name == player[0]:
                player.append(id)

def get_player_stats_by_id(id):
    url = "https://api.pbpstats.com/get-all-season-stats/nba"
    params = {
        "EntityType": "Player",
        "EntityId": id
    }
    player_stats = requests.get(url, params=params).json()

    for json_object in player_stats["results"][game_type]:
        for key, value in json_object.items():
            if key == "Season" and value == game_season:
                return json_object


def get_team_stats(team):
    for player in team:
        all_stats = get_player_stats_by_id(player[2])
        for key in key_all_stats:
            if key in all_stats:
                player.append(all_stats[key])
            else:
                player.append(None)
        # Add player position
        # player.append(get_player_position(player[2]))

def get_player_position(player_id):
    from nba_api.stats.endpoints import commonplayerinfo
    player_info = commonplayerinfo.CommonPlayerInfo(player_id)
    player_info = player_info.get_normalized_dict()
    return player_info["CommonPlayerInfo"][0]["POSITION"]

# Print messages
col_names = ["Name", "Minutes", "ID"] + key_all_stats
def print_table(team):
    print(tabulate(team, headers=col_names, tablefmt="grid"))

## Away team stats:

In [38]:
# Away team stats
print("Away team players:")
away_team_players_data = {}
away_team = get_team_players(game_stats, "Away")
get_player_ids(away_team, "Away")
get_team_stats(away_team)
print_table(away_team)

Away team players:
+-------------------+-----------+--------+---------------+----------+----------+------------+-------+-------------+-------------+----------+----------+---------+
| Name              | Minutes   |     ID |   GamesPlayed |   Fg2Pct |   Fg3Pct |   FtPoints |   FTA |   Turnovers |   TotalPoss |   Steals |   Blocks |   Fouls |
+===================+===========+========+===============+==========+==========+============+=======+=============+=============+==========+==========+=========+
| Stephen Curry     | 37:32     | 201939 |            78 | 0.449239 | 0.453333 |        262 |   291 |         240 |       11797 |      126 |       12 |     198 |
+-------------------+-----------+--------+---------------+----------+----------+------------+-------+-------------+-------------+----------+----------+---------+
| Klay Thompson     | 34:26     | 202691 |            82 | 0.437408 | 0.401141 |        132 |   157 |         157 |       11576 |       86 |       45 |     237 |
+--------

## Home team stats:

In [8]:
# Home team stats
print("Home team players:")
home_team = get_team_players(game_stats, "Home")
get_player_ids(home_team, "Home")
get_team_stats(home_team)
print_table(home_team)

Home team players:
+-----------------+-----------+--------+---------------+----------+----------+------------+-------+-------------+--------------------+----------+----------+---------+
| Name            | Minutes   |     ID |   GamesPlayed |   Fg2Pct |   Fg3Pct |   FtPoints |   FTA |   Turnovers |   BadPassTurnovers |   Steals |   Blocks |   Fouls |
+=================+===========+========+===============+==========+==========+============+=======+=============+====================+==========+==========+=========+
| Al Jefferson    | 31:34     |   2744 |            78 | 0.49959  | 0.117647 |        167 |   217 |         105 |                 31 |       80 |       89 |     170 |
+-----------------+-----------+--------+---------------+----------+----------+------------+-------+-------------+--------------------+----------+----------+---------+
| Paul Millsap    | 30:31     | 200794 |            78 | 0.497608 | 0.333333 |        264 |   355 |         139 |                 38 |      102 | 

## Aggregate and export the key stats to CSV

In [41]:
# Get the top 5 players key stats
away_team_stats = get_team_players(game_stats, "Away")
get_player_ids(away_team_stats, "Away")
get_team_stats(away_team_stats)
col_names = ["Name", "Minutes", "ID"] + key_all_stats
# Convert the away_team_stats to a list of players which is a dictionary with the cols as keys
away_team_stats = [{col_names[i]: player[i] for i in range(len(col_names))} for player in away_team_stats]
# Add the player position to the dictionary
for player in away_team_stats:
    player["Position"] = get_player_position(player["ID"])
# Add the pass success rate which is the opposite of turnovers per possession to the dictionary
for player in away_team_stats:
    player["PassRatio"] = 1 - player["Turnovers"] / player["TotalPoss"]

# Print the top 5 players
print("Top 5 players:")
print(tabulate(away_team_stats[:5], headers="keys", tablefmt="grid"))

Top 5 players:
+-----------------+-----------+--------+---------------+----------+----------+------------+-------+-------------+-------------+----------+----------+---------+----------------+--------------------+
| Name            | Minutes   |     ID |   GamesPlayed |   Fg2Pct |   Fg3Pct |   FtPoints |   FTA |   Turnovers |   TotalPoss |   Steals |   Blocks |   Fouls | Position       |   TurnoversPerPoss |
+=================+===========+========+===============+==========+==========+============+=======+=============+=============+==========+==========+=========+================+====================+
| Stephen Curry   | 37:32     | 201939 |            78 | 0.449239 | 0.453333 |        262 |   291 |         240 |       11797 |      126 |       12 |     198 | Guard          |          0.0203442 |
+-----------------+-----------+--------+---------------+----------+----------+------------+-------+-------------+-------------+----------+----------+---------+----------------+-----------------